In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer,TFBertModel,TFBertForSequenceClassification
import fasttext
import spacy
import tensorflow_hub as hub
from transformers import TFAutoModel, AutoTokenizer


In [2]:
test_csv = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv')
train_csv=pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')
sample_=pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/sample_submission.csv')
misconsepction=pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')

In [3]:
"merhaba"

'merhaba'

In [4]:
def get_correct_answer_text(row):
    answer_columns = {
        'A': 'AnswerAText',
        'B': 'AnswerBText',
        'C': 'AnswerCText',
        'D': 'AnswerDText'
    }
    # Retrieve the correct column and return its value for the current row
    correct_column = answer_columns.get(row['CorrectAnswer'])

    return row[correct_column] if correct_column else None

# Create the CorrectAnswerText column by applying the function row by row
train_csv['CorrectAnswerText'] = train_csv.apply(get_correct_answer_text, axis=1)

In [5]:
def merged_processes(main_csv,istenc,col_name):
    
    gayri_resmi=[]
    
    for i,k in main_csv[istenc].items():
        if(k.is_integer()):
            gayri_resmi.append(misconsepction.iloc[int(k)].MisconceptionName)
        else:
            gayri_resmi.append(np.nan)
    main_csv[col_name]=gayri_resmi   
    return main_csv
train_csv=merged_processes(train_csv,"MisconceptionAId","a_missc")
train_csv=merged_processes(train_csv,"MisconceptionBId","b_missc")
train_csv=merged_processes(train_csv,"MisconceptionCId","c_missc")
train_csv=merged_processes(train_csv,"MisconceptionDId","d_missc")

In [6]:
train_csv

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,CorrectAnswerText,a_missc,b_missc,c_missc,d_missc
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,\( 3 \times(2+4)-5 \),NaN,NaN,NaN,"Confuses the order of operations, believes add..."
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,Does not simplify,Does not know that to factorise a quadratic ex...,Thinks that when you cancel identical terms fr...,Does not know that to factorise a quadratic ex...,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,Only\nKatie,Believes if you changed all values by the same...,NaN,Believes if you changed all values by the same...,Believes if you add the same value to all numb...
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,\( 90^{\circ} \),Does not know the properties of a rectangle,Does not know the properties of a rectangle,NaN,Does not know the properties of a rectangle
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,\( 30 \),NaN,NaN,NaN,Thinks you can find missing values in a given ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1864,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,C,What is the range of the following numbers?\n\...,\( 5 \),\( 11 \),\( 23 \),\( 16 \),2456.0,691.0,NaN,1349.0,\( 23 \),Finds the mode rather than the range,To find the range adds the biggest and smalles...,NaN,Does not include negative numbers when calcula...
1865,1865,2695,"Describe an enlargement, with no centre of enl...",90,Length Scale Factors in Similar Shapes,B,Shape \( Q \) is an enlargement of shape \( P ...,\( 3 \div 11 \),\( 11 \div 3 \),\( 3 \times 11 \),\( 11-3 \),1500.0,NaN,2442.0,1258.0,\( 11 \div 3 \),"When calculating a scale factor, thinks the di...",NaN,"When calculating a scale factor, multiplies in...","When calculating a scale factor, thinks they n..."
1866,1866,854,Use the order of operations to carry out calcu...,33,BIDMAS,B,What does the following equal?\n\[\n8-7+10 \ti...,\( 36 \),\( 31 \),\( -29 \),\( 33 \),NaN,NaN,2306.0,1507.0,\( 31 \),NaN,NaN,Applies BIDMAS in strict order (does not reali...,Carries out operations from left to right rega...
1867,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Tom and Katie are discussing congruence and si...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,2312.0,NaN,2312.0,2312.0,Only Katie,Thinks congruent shapes are the same shape but...,NaN,Thinks congruent shapes are the same shape but...,Thinks congruent shapes are the same shape but...


In [7]:
rows = []
for idx, row in train_csv.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text =f"#question:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n #Correct Answer: {correct_answer}\n #Misconcepte Incorrect answer###:{option}.{row[f'Answer{option}Text']}"

        rows.append({"query_text": query_text, 
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     "ConstructName": row.ConstructName,
                     "SubjectName": row.SubjectName,
                     "QuestionText": row.QuestionText,
                     "correct_answer": correct_answer,
                     "incorrect_answer": row[f"Answer{option}Text"],
                     "miss": misconsepction.iloc[int(row[f"Misconception{option}Id"])].MisconceptionName if row[f"Misconception{option}Id"].is_integer() else np.nan,
                     "missid":row[f"Misconception{option}Id"] if row[f"Misconception{option}Id"].is_integer() else np.nan
                     })
        

df = pd.DataFrame(rows)
df

,query_text,QuestionId_Answer,ConstructName,SubjectName,QuestionText,correct_answer,incorrect_answer,miss,missid
0,#question:BIDMAS-Use the order of operations t...,0_B,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),NaN,NaN
1,#question:BIDMAS-Use the order of operations t...,0_C,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times(2+4-5) \),NaN,NaN
2,#question:BIDMAS-Use the order of operations t...,0_D,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),Does not need brackets,"Confuses the order of operations, believes add...",1672.0
3,#question:Simplifying Algebraic Fractions-Simp...,1_A,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+1 \),Does not know that to factorise a quadratic ex...,2142.0
4,#question:Simplifying Algebraic Fractions-Simp...,1_B,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+2 \),Thinks that when you cancel identical terms fr...,143.0
...,...,...,...,...,...,...,...,...,...
5602,#question:Congruency in Other Shapes-Distingui...,1867_C,Distinguish between congruency and similarity,Congruency in Other Shapes,Tom and Katie are discussing congruence and si...,Only Katie,Both Tom and Katie,Thinks congruent shapes are the same shape but...,2312.0
5603,#question:Congruency in Other Shapes-Distingui...,1867_D,Distinguish between congruency and similarity,Congruency in Other Shapes,Tom and Katie are discussing congruence and si...,Only Katie,Neither is correct,Thinks congruent shapes are the same shape but...,2312.0
5604,#question:Rotation-Describe a 90° or 270° rota...,1868_A,Describe a 90° or 270° rotation giving the ang...,Rotation,Jo and Paul are arguing about how to fully des...,Only Paul,Only\nJo,Forgets to give the centre of rotation,801.0
5605,#question:Rotation-Describe a 90° or 270° rota...,1868_C,Describe a 90° or 270° rotation giving the ang...,Rotation,Jo and Paul are arguing about how to fully des...,Only Paul,Both Jo and Paul,Forgets to give the centre of rotation,801.0


In [8]:
df=df.dropna(subset=["miss"])

In [9]:
import re
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub("@\w+", '',x)      # Delete strings starting with @
    #x = re.sub("'\d+", '',x)      # Delete Numbers
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\times+", "\\\\times", x)
    x=re.sub(r"\°+","degree",x)
   
    # Replace 'times' with '\times' with spaces around
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

In [10]:
new_df=pd.DataFrame(df["query_text"].apply(preprocess_text))

In [11]:
new_df

,query_text
2,#question:bidmas-use the order of operations t...
3,#question:simplifying algebraic fractions-simp...
4,#question:simplifying algebraic fractions-simp...
5,#question:simplifying algebraic fractions-simp...
6,#question:range and interquartile range from a...
...,...
5602,#question:congruency in other shapes-distingui...
5603,#question:congruency in other shapes-distingui...
5604,#question:rotation-describe a 90degree or 270d...
5605,#question:rotation-describe a 90degree or 270d...


In [12]:
onehot=pd.get_dummies(df["missid"]).astype("int")

In [13]:
type(onehot)

pandas.core.frame.DataFrame

In [14]:
#Are we working on Gpu.lets Check in
tf.config.list_physical_devices(device_type="GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [15]:
model_name = "/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

print("Model başarıyla yüklendi!")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model başarıyla yüklendi!


In [16]:
tokens = tokenizer(
    list(new_df["query_text"]),
    padding=True,          # Doldurma işlemi
    truncation=True,
    max_length=128,# Uzun metinleri kesme
    return_tensors="tf"    # TensorFlow tensor formatında çıktı
)
batch_size = 32  # Belirli bir batch boyutu seçin
sentence_embeddings = []
for i in range(0, len(new_df["query_text"]), batch_size):
    batch_texts = list(new_df["query_text"])[i:i + batch_size]
    tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="tf")
    outputs = model(**tokens)
    embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1)
    sentence_embeddings.append(embeddings)

# Grupları birleştir
sentence_embeddings = tf.concat(sentence_embeddings, axis=0)


In [17]:
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
def a_little_bit_cosine_similarity():
    normalized_vectors=normalize(sentence_embeddings)
    
    def making_cosine_per_triple():
        a=0
        liste=[]
        while(a<len(normalized_vectors)):
            try:
                elements=normalized_vectors[a:a+3]
                one_two=cosine_similarity([elements[0]],[elements[1]])
                two_three=cosine_similarity([elements[1]],[elements[2]])
                one_three=cosine_similarity([elements[0]],[elements[2]])
                compare=[one_two,two_three,one_three]
                liste.append(compare)
                a+=3
            except :
                liste.append(-1)
                a+=3
        return liste    
    result=making_cosine_per_triple()
    return result
funct=a_little_bit_cosine_similarity()
funct

[[array([[0.73502293]]), array([[0.99517712]]), array([[0.73821337]])],
 [array([[0.59400017]]), array([[0.98089925]]), array([[0.5986697]])],
 [array([[0.68444347]]), array([[0.99436792]]), array([[0.68524776]])],
 [array([[0.64059569]]), array([[0.71766788]]), array([[0.74298014]])],
 [array([[0.99203896]]), array([[0.75577291]]), array([[0.77194871]])],
 [array([[0.98710954]]), array([[0.87694474]]), array([[0.87842083]])],
 [array([[0.9964382]]), array([[0.76948004]]), array([[0.76744647]])],
 [array([[0.99924711]]), array([[0.59593282]]), array([[0.58958131]])],
 [array([[0.99189613]]), array([[0.28572538]]), array([[0.27757397]])],
 [array([[0.99537738]]), array([[0.99340892]]), array([[0.99234873]])],
 [array([[0.99635294]]), array([[0.90810707]]), array([[0.90535863]])],
 [array([[0.50698738]]), array([[0.99642539]]), array([[0.47856322]])],
 [array([[0.46876319]]), array([[0.99303022]]), array([[0.46832538]])],
 [array([[0.9985239]]), array([[0.99895708]]), array([[0.99853057]

In [18]:
normalized_vectors=normalize(sentence_embeddings)

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(normalized_vectors,onehot,test_size=0.15)
x_train_stacked=np.stack(x_train)
x_valid_stacked=np.stack(x_valid)



In [20]:
def create_model():
    model = tf.keras.Sequential([
        
        tf.keras.layers.Dense(256, activation='relu',input_shape=(384,)),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Dense(128, activation='relu'), 
        
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64,activation="relu"),
         tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
         tf.keras.layers.Dense(32,activation="relu"),
         
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1604, activation='softmax')                      
    ])
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )
    return model
def early_stopping_for_valids():
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                  patience=15)
        return early_stopping    

In [21]:
def train_model_f_test(epoch=200):
         model=create_model()
         
         #lir=tf.keras.callbacks.LearningRateScheduler(self.lr_schedule)
         model.fit(x_train_stacked,y_train,epochs=epoch, validation_data=(x_valid_stacked, y_valid),callbacks=[early_stopping_for_valids()])
         return model

clfier=train_model_f_test()

Epoch 1/200


I0000 00:00:1733657179.892778      68 service.cc:145] XLA service 0x7f99c80e3470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733657179.892829      68 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733657179.892833      68 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1733657180.119059      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


117/117 [==============================] - 19s 13ms/step - loss: 7.3861 - accuracy: 0.0011 - val_loss: 7.3806 - val_accuracy: 0.0000e+00
Epoch 2/200
117/117 [==============================] - 1s 6ms/step - loss: 7.3324 - accuracy: 0.0043 - val_loss: 7.3793 - val_accuracy: 0.0000e+00
Epoch 3/200
117/117 [==============================] - 1s 6ms/step - loss: 7.2856 - accuracy: 0.0100 - val_loss: 7.3712 - val_accuracy: 0.0030
Epoch 4/200
117/117 [==============================] - 1s 6ms/step - loss: 7.2449 - accuracy: 0.0162 - val_loss: 7.3481 - val_accuracy: 0.0122
Epoch 5/200
117/117 [==============================] - 1s 6ms/step - loss: 7.1997 - accuracy: 0.0374 - val_loss: 7.3025 - val_accuracy: 0.0305
Epoch 6/200
117/117 [==============================] - 1s 6ms/step - loss: 7.1613 - accuracy: 0.0471 - val_loss: 7.2445 - val_accuracy: 0.0473
Epoch 7/200
117/117 [==============================] - 1s 6ms/step - loss: 7.1160 - accuracy: 0.0660 - val_loss: 7.1969 - val_accuracy: 0.0671
E

In [22]:
test_csv

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets
1,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify
2,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct


In [23]:
def get_correct_answer_text(row):
    answer_columns = {
        'A': 'AnswerAText',
        'B': 'AnswerBText',
        'C': 'AnswerCText',
        'D': 'AnswerDText'
    }
    # Retrieve the correct column and return its value for the current row
    correct_column = answer_columns.get(row['CorrectAnswer'])

    return row[correct_column] if correct_column else None
test_csv["CorrectAnswerText"]=test_csv.apply(get_correct_answer_text,axis=1)

rows = []
for idx, row in test_csv.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text =f"#question:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n #Correct Answer: {correct_answer}\n #Misconcepte Incorrect answer###:{option}.{row[f'Answer{option}Text']}"
        rows.append({"query_text": query_text, 
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     "ConstructName": row.ConstructName,
                     "SubjectName": row.SubjectName,
                     "QuestionText": row.QuestionText,
                     "correct_answer": correct_answer,
                     "incorrect_answer": row[f"Answer{option}Text"],
                     
                     })
        

test_df = pd.DataFrame(rows)
import re
def preprocess_text(x):
    x = x.lower()                 # Convert words to lowercase
    x = re.sub("@\w+", '',x)      # Delete strings starting with @
    #x = re.sub("'\d+", '',x)      # Delete Numbers
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\times+", "\\\\times", x)
    x=re.sub(r"\°+","degree",x)
    
    # Replace 'times' with '\times' with spaces around
    x = x.strip()        
    return x



new_df_test=pd.DataFrame(test_df["query_text"].apply(preprocess_text))


In [24]:
tokens = tokenizer(
    list(new_df_test["query_text"]),
    padding=True,          # Doldurma işlemi
    truncation=True,
    max_length=128,# Uzun metinleri kesme
    return_tensors="tf" ,
    
)
outputs_test = model(**tokens)

# Cümle temsillerinin havuzlama işlemi
sentence_embeddings_test = tf.reduce_mean(outputs_test.last_hidden_state, axis=1)

print("Cümle Temsilleri (Shape):", sentence_embeddings_test.shape)
print(sentence_embeddings_test)

Cümle Temsilleri (Shape): (9, 384)
tf.Tensor(
[[-0.23226526  0.21983854  0.2265437  ...  0.06166925 -0.14788291
  -0.05995538]
 [-0.23168077  0.22544843  0.23693112 ...  0.0398023  -0.12510137
  -0.0597554 ]
 [-0.25819266  0.23460309  0.2184826  ...  0.10904375 -0.10805874
  -0.06689815]
 ...
 [-0.01112169  0.20174749  0.13211995 ... -0.09591085 -0.00484345
   0.13340095]
 [-0.01112169  0.20174749  0.13211995 ... -0.09591085 -0.00484345
   0.13340095]
 [-0.01112169  0.20174749  0.13211995 ... -0.09591085 -0.00484345
   0.13340095]], shape=(9, 384), dtype=float32)


In [25]:
predicts_test=clfier.predict(sentence_embeddings_test)

1/1 [==============================] - 0s 461ms/step


In [26]:
top_list_main=[]
index_mm=[]
for i in predicts_test:
    
    sortede=sorted(i, reverse=True)[:25]
     
    print(sortede)
    top_list_main.append(sortede)
    index_topluluk=[]
    for k in sortede:
        
        index_num=list(i).index(k)
        index_topluluk.append(index_num)
    index_mm.append(index_topluluk)

[0.3164949, 0.1411646, 0.089855984, 0.08619266, 0.086099826, 0.085158266, 0.0760763, 0.028822148, 0.010223422, 0.008443865, 0.008260386, 0.006574897, 0.0036887953, 0.0035828738, 0.0034572284, 0.003208373, 0.0031520328, 0.0022696701, 0.002259914, 0.0022432194, 0.0021134177, 0.002013474, 0.0018075417, 0.001779576, 0.0014997027]
[0.60226977, 0.1154422, 0.08135382, 0.073257856, 0.03599194, 0.024221778, 0.008513315, 0.007922879, 0.0072021373, 0.004531864, 0.0043612695, 0.0036979672, 0.003374988, 0.002346846, 0.0018192253, 0.0015144951, 0.0014811341, 0.0012742593, 0.0010746283, 0.0010201173, 0.00094765826, 0.00090795074, 0.0008835632, 0.00078936055, 0.00074873766]
[0.29796156, 0.2322812, 0.13743633, 0.135937, 0.06413095, 0.028644448, 0.020959526, 0.018783232, 0.0152128935, 0.011344014, 0.0051418906, 0.0050225873, 0.003628788, 0.0035004707, 0.002906978, 0.0021775686, 0.0018699896, 0.0017233231, 0.0016444958, 0.0013646708, 0.0011322985, 0.00088485086, 0.00076945825, 0.0006801295, 0.0006159366]

In [27]:
pd.DataFrame(top_list_main)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.316495,0.141165,0.089856,0.086193,0.086100,0.085158,0.076076,0.028822,0.010223,0.008444,...,0.003208,0.003152,0.002270,0.002260,0.002243,0.002113,0.002013,0.001808,0.001780,1.499703e-03
1,0.602270,0.115442,0.081354,0.073258,0.035992,0.024222,0.008513,0.007923,0.007202,0.004532,...,0.001514,0.001481,0.001274,0.001075,0.001020,0.000948,0.000908,0.000884,0.000789,7.487377e-04
2,0.297962,0.232281,0.137436,0.135937,0.064131,0.028644,0.020960,0.018783,0.015213,0.011344,...,0.002178,0.001870,0.001723,0.001644,0.001365,0.001132,0.000885,0.000769,0.000680,6.159366e-04
3,0.862183,0.064923,0.054254,0.009478,0.005787,0.001014,0.000754,0.000568,0.000227,0.000205,...,0.000031,0.000023,0.000020,0.000018,0.000012,0.000007,0.000005,0.000005,0.000003,3.036403e-06
4,0.923470,0.045111,0.022835,0.006197,0.001565,0.000329,0.000221,0.000051,0.000041,0.000033,...,0.000011,0.000006,0.000005,0.000005,0.000004,0.000002,0.000002,0.000001,0.000001,9.263695e-07
5,0.805226,0.152278,0.032061,0.007225,0.000787,0.000683,0.000563,0.000401,0.000275,0.000122,...,0.000016,0.000015,0.000012,0.000008,0.000004,0.000004,0.000004,0.000004,0.000004,2.308501e-06
6,0.368835,0.308960,0.069786,0.038827,0.036368,0.021640,0.015063,0.012839,0.006754,0.006657,...,0.003753,0.003699,0.003354,0.003323,0.003187,0.003143,0.002977,0.002706,0.002169,1.987887e-03
7,0.368835,0.308960,0.069786,0.038827,0.036368,0.021640,0.015063,0.012839,0.006754,0.006657,...,0.003753,0.003699,0.003354,0.003323,0.003187,0.003143,0.002977,0.002706,0.002169,1.987887e-03
8,0.368835,0.308960,0.069786,0.038827,0.036368,0.021640,0.015063,0.012839,0.006754,0.006657,...,0.003753,0.003699,0.003354,0.003323,0.003187,0.003143,0.002977,0.002706,0.002169,1.987887e-03


In [28]:
liste_index=pd.DataFrame(index_mm)

In [29]:
liste=[]
for i in range(9):
    liste.append(sorted(liste_index.iloc[i]))
    
pd.set_option('display.max_colwidth', None)

main_list=pd.DataFrame(liste)

def row_stringer(row):
    
    result=" ".join(map(str, list(row)))
    return result


simple=np.array(main_list.apply(row_stringer,axis=1))
simple

array(['96 141 244 273 326 394 449 497 554 584 652 723 746 823 907 948 975 1071 1260 1271 1389 1403 1415 1541 1549',
       '96 244 273 326 394 449 497 546 554 584 613 652 723 823 907 948 975 1071 1260 1271 1360 1389 1403 1541 1549',
       '92 144 244 273 326 394 449 497 584 613 622 642 723 739 823 839 907 948 975 1071 1271 1389 1415 1486 1549',
       '36 66 94 133 170 261 280 292 494 497 716 952 973 979 1104 1169 1170 1174 1245 1279 1403 1473 1513 1518 1600',
       '36 92 94 133 261 292 384 494 497 716 721 952 973 979 1104 1169 1170 1174 1193 1279 1403 1473 1513 1518 1600',
       '36 66 133 261 292 341 494 497 642 716 721 917 950 952 979 1104 1169 1170 1174 1279 1403 1473 1513 1518 1600',
       '48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 821 838 854 994 1004 1063 1471 1477 1529 1537',
       '48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 821 838 854 994 1004 1063 1471 1477 1529 1537',
       '48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 

In [30]:
rows = []
for idx, row in test_csv.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        

        rows.append({
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     
                     
                     })
        

predddf = pd.DataFrame(rows)
predddf["MisconceptionId"]=simple


In [31]:
predddf.MisconceptionId.iloc[0][2]

' '

In [32]:
predddf.to_csv("submission.csv" ,index=False)

In [33]:
pd.read_csv("submission.csv")

,QuestionId_Answer,MisconceptionId
0,1869_B,96 141 244 273 326 394 449 497 554 584 652 723 746 823 907 948 975 1071 1260 1271 1389 1403 1415 1541 1549
1,1869_C,96 244 273 326 394 449 497 546 554 584 613 652 723 823 907 948 975 1071 1260 1271 1360 1389 1403 1541 1549
2,1869_D,92 144 244 273 326 394 449 497 584 613 622 642 723 739 823 839 907 948 975 1071 1271 1389 1415 1486 1549
3,1870_A,36 66 94 133 170 261 280 292 494 497 716 952 973 979 1104 1169 1170 1174 1245 1279 1403 1473 1513 1518 1600
4,1870_B,36 92 94 133 261 292 384 494 497 716 721 952 973 979 1104 1169 1170 1174 1193 1279 1403 1473 1513 1518 1600
5,1870_C,36 66 133 261 292 341 494 497 642 716 721 917 950 952 979 1104 1169 1170 1174 1279 1403 1473 1513 1518 1600
6,1871_A,48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 821 838 854 994 1004 1063 1471 1477 1529 1537
7,1871_C,48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 821 838 854 994 1004 1063 1471 1477 1529 1537
8,1871_D,48 104 361 377 452 566 660 667 670 735 742 754 792 796 803 821 838 854 994 1004 1063 1471 1477 1529 1537


In [34]:
print("Paketler Tamam")

Paketler Tamam
